### Data

In [ ]:
import numpy as np
import cv2
import os
from scipy import ndimage
from tqdm import tqdm
import keras
from keras.layers import Input, Conv2D, Conv2DTranspose, Reshape, Flatten, Dense, Lambda
from keras.models import Model
import keras.backend as K
import tensorflow as tf
from keras.losses import mse

from sklearn import metrics as sklearn_metrics
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors

from google.colab.patches import cv2_imshow
from google.colab import drive

import shutil

In [ ]:
drive.mount('/content/drive')

In [ ]:
!mkdir /content/images
!mkdir /content/images/test
!mkdir /content/labels
!mkdir /content/labels/test
!mkdir /content/extra_labels
!mkdir /content/extra_labels/test
!mkdir /content/rotations
!mkdir /content/rotations/test

In [ ]:
!unzip -q '/content/drive/My Drive/NATO/test6.zip' -d '/content/images/test'
!unzip -q '/content/drive/My Drive/NATO/test6l.zip' -d '/content/labels/test'
!unzip -q '/content/drive/My Drive/NATO/test6el.zip' -d '/content/extra_labels/test'
!unzip -q '/content/drive/My Drive/NATO/test6r.zip' -d '/content/rotations/test'

### Model

In [ ]:
def Sampling(inputs):
    mean, log_var = inputs
    return K.random_normal(tf.shape(mean)) * K.exp(log_var / 2) + mean 
    
def model_version12():
    # autoencoder model
    beta_value = 50
    # encoder
    input = Input(shape=(80, 80, 1))
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(input)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    codings_mean = keras.layers.Dense(10, name="encoder_mean")(x)
    codings_log_var = keras.layers.Dense(10, name="encoder_log_var")(x)
    codings = Lambda(Sampling, name="encoder_output")([codings_mean, codings_log_var])
    variational_encoder = keras.models.Model(inputs=[input], outputs=[codings_mean, codings_log_var, codings], name="encoder")

    #decoder
    decoder_inputs = keras.layers.Input(shape=(10))
    z = Dense(256, activation='relu')(decoder_inputs)
    z = Dense(256, activation='relu')(z)
    z = Dense(5*5*32, activation='relu')(z)
    z = Reshape((5, 5, 32))(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    output = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(z)
    variational_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[output], name="decoder")

    _, _, codings = variational_encoder(input)
    reconstructions = variational_decoder(codings)
    variational_ae = keras.models.Model(inputs=[input], outputs=[reconstructions], name="autoencoder")

    reconstruction_loss_factor = 1000
    reconstruction_loss = mse(K.flatten(input), K.flatten(reconstructions))
    reconstruction_loss *= 80 * 80 * 1
    kl_loss = -0.5 * beta_value * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=1) 
    vae_loss = K.mean(reconstruction_loss_factor * reconstruction_loss + kl_loss)
    variational_ae.add_loss(vae_loss)

    variational_ae.add_metric(kl_loss, name="kl_loss")
    variational_ae.add_metric(reconstruction_loss, name="reconstruction_loss")

    return variational_ae

### Validation

In [ ]:
Dict = {0: 'advance_to_contact', 2: 'attack', 9: 'counterattack', 18: 'main_attack'}
Dict2 = {0: 'otse_k', 1: 'otse_l', 2: 'parem_n', 3: 'parem_y', 4: 'vasak_n', 5: 'vasak_y'}
Dict3 = {"otse_l": 0, "otse_k": 1, "parem_n": 2, "parem_y": 3, "vasak_n": 4, "vasak_y": 5}

In [ ]:
images_test = '/content/images/test'
labels_test = '/content/labels/test'
extra_labels_test = '/content/extra_labels/test'
rotations_test = '/content/rotations/test'

file_list = os.listdir(images_test)
model = model_version12()
model2 = model_version12()
model3 = model_version12()
model4 = model_version12()

actual = []
liPredicted = []

symbolType = "advance_to_contact"

model.load_weights('/content/drive/MyDrive/models_trajectory_final/advance_to_contact_autoencoder_model12_b50_10.h5')
encoder_atc = Model(inputs=model.input, outputs=model.get_layer("encoder").output)

model2.load_weights('/content/drive/MyDrive/models_trajectory_final/attack_autoencoder_model12_b50_10.h5')
encoder_a = Model(inputs=model2.input, outputs=model2.get_layer("encoder").output)

model3.load_weights('/content/drive/MyDrive/models_trajectory_final/counterattack_autoencoder_model12_b50_10.h5')
encoder_ca = Model(inputs=model3.input, outputs=model3.get_layer("encoder").output)

model4.load_weights('/content/drive/MyDrive/models_trajectory_final/main_attack_autoencoder_model12_b50_10.h5')
encoder_ma = Model(inputs=model4.input, outputs=model4.get_layer("encoder").output)

for name in tqdm(file_list):
    image = cv2.imread(images_test+"/"+name, cv2.IMREAD_GRAYSCALE)
    imageW = image.shape[1]
    imageH = image.shape[0]
    with open(labels_test+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]

    with open(extra_labels_test+"/"+str(name).split(".")[0]+".txt") as f:
        lines_extra = [line.rstrip('\n') for line in f]

    with open(rotations_test+"/"+str(name).split(".")[0]+".txt") as f:
        rotations = [line.rstrip('\n') for line in f]

        for i, line in enumerate(lines):
            cls = int(line.split(" ")[0])
            # if Dict[cls] == symbolType:
            
            x = int(((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * imageW) - 5)
            y = int(((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * imageH) - 5)
            w = int((float(line.split(" ")[3]) * imageW) + 5)
            h = int((float(line.split(" ")[4]) * imageH) + 5)

            # Symbol
            crop_image = image[y:y+h, x:x+w]
            img_rotated = ndimage.rotate(crop_image, -int(rotations[i]), mode='constant', cval=255)
            dim = (80, 80)
            resized_img = cv2.resize(img_rotated, dim, interpolation = cv2.INTER_AREA)
            binImg = img = np.where(resized_img <= 150, 0, 255)

            ## Predict
            formatImage = np.array([binImg])
            formatImage = formatImage.astype('float32') / 255.
            formatImage = formatImage.reshape(formatImage.shape[0], formatImage.shape[1], formatImage.shape[2], 1)

            if Dict[cls] == "advance_to_contact":
                _, _, predicted = encoder_atc.predict(formatImage, verbose = 0)
            elif Dict[cls] == "attack":
                _, _, predicted = encoder_a.predict(formatImage, verbose = 0)
            elif Dict[cls] == "counterattack":
                _, _, predicted = encoder_ca.predict(formatImage, verbose = 0)
            elif Dict[cls] == "main_attack":
                _, _, predicted = encoder_ma.predict(formatImage, verbose = 0)
            else:
                continue


            dataLoad = np.load("/content/drive/MyDrive/npy_final/"+Dict[cls]+"_data_model12_b50_10.npy", allow_pickle=True)

            load_features = dataLoad.item().get("features")
            load_images = dataLoad.item().get("images")
            load_extra_class = dataLoad.item().get("extra_label")
            features_reshape = load_features.reshape((-1, np.prod((load_features.shape[1:]))))

            knn_cosine = NearestNeighbors(n_neighbors=1, metric="cosine")
            knn_cosine.fit(features_reshape)

            actual.append(int(lines_extra[i]))

            predicted_reshape = predicted.reshape((-1, np.prod((predicted.shape[1:]))))
            _, indices = knn_cosine.kneighbors([predicted_reshape[0]])
            result = [load_extra_class[idx] for idx in indices.flatten()]
            result_img = [load_images[idx] for idx in indices.flatten()]
            liPredicted.append(Dict3[result[0]])

In [ ]:
confusion_matrix = sklearn_metrics.confusion_matrix(actual, liPredicted)
cm_display = sklearn_metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['wide s.', 'narrow s.', '45° right', '90° right', '45° left', '90° left'])

fig, ax = plt.subplots(figsize=(5,5))
cm_display.plot(ax = ax, xticks_rotation = 45)

print("Accuracy:", accuracy_score(actual, liPredicted))